## Error propagation analysis for Fixed point arithmetic using fxpmath

In this notebook, we use library fxpmath to analyze the propagation of errors due to fixed point arithmetic for algorithms described in [Compilation of Fault-Tolerant Quantum Heuristics for Combinatorial Optimization](https://arxiv.org/abs/2007.07391). 

Qualtran should provide tools to enable such analysis for algorithms that use fixed point arithmetic Bloqs as subroutines. 

In [ ]:
from typing import Optional
from fxpmath import Fxp
import numpy as np

def assert_allclose(x: Fxp, y: float, eps: float):
    np.testing.assert_allclose(x.get_val(), y, atol=eps)

###  Appendix D4: Multiplying an integer to a real number
Goal: Given quantum registers A and B with real number $\kappa$ ($0 \leq \kappa \lt 1$) and a $d_{B}$-bit integer $\lambda$, our goal is to compute an approximation $\widetilde{\gamma}$ of $\gamma = \kappa * \lambda$ s.t. $|\widetilde{\gamma} - \gamma| < \epsilon$

$$
    |\kappa\rangle_{A} |\lambda\rangle_{B} |0\rangle_{\text{out}} \rightarrow |\kappa\rangle_{A} |\lambda\rangle_{B} |\widetilde{\gamma}\rangle_{\text{out}}
$$


We analyze the error due to fixed point arithmetic for algorithm described in Appendix D4 of https://arxiv.org/abs/2007.07391

In [ ]:
from qualtran import val_to_fxp
from qualtran.bloqs.arithmetic.multiplication import ScaleIntByReal
# Multiplying a real numbers with an d_B-bit integer.
def get_bitsize_for_fxp_mul_with_integer(eps: float, d_B: int):
    return d_B + int(np.ceil(np.log2(d_B / eps))) # Equation D7

def test_multiplication_with_integer_for_eps(eps: float, d_B: int, d_A: int):
    rng = np.random.default_rng(int(eps * d_B * 1e9))
    try:
        for _ in range(100):
            a, = rng.random(1)
            b = rng.integers(0, 1 << d_B)
            res = ScaleIntByReal(r_bitsize=d_A, i_bitsize=d_B).on_classical_vals(**{'real_in': a, 'int_in': b})
            assert_allclose(res['result'], a * b, eps)
        print(f'Success! {eps=}, {d_A=}, {d_B=}')
    except AssertionError:
        print(f'Failed! {eps=}, {d_A=}, {d_B=}')

for eps in [1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-9]:
    for d_B in [5, 8, 11]:
        d_A = get_bitsize_for_fxp_mul_with_integer(eps, d_B)
        test_multiplication_with_integer_for_eps(eps, d_B, d_A)

###  Appendix D5: Multiplying two different real numbers
Goal: Given quantum registers A and B with real numbers $\kappa$ ($0 \leq \kappa \lt 1$) and $\lambda$ ($0 \leq \lambda \lt 1$), our goal is to compute an approximation $\widetilde{\gamma}$ of $\gamma = \kappa * \lambda$ s.t. $|\widetilde{\gamma} - \gamma| < \epsilon$

$$
    |\kappa\rangle_{A} |\lambda\rangle_{B} |0\rangle_{\text{out}} \rightarrow |\kappa\rangle_{A} |\lambda\rangle_{B} |\widetilde{\gamma}\rangle_{\text{out}}
$$


We analyze the error due to fixed point arithmetic for algorithm described in Appendix D5 of https://arxiv.org/abs/2007.07391

In [ ]:
# Multiplying two real numbers
from qualtran.bloqs.arithmetic.multiplication import MultiplyTwoReals


def get_bitsize_for_fxp_mul(eps: float):
    return int(np.ceil(1 + np.log2(1/eps)) + np.log2(1 + np.log2(1/eps))) # Equation D17

def test_multiplication_for_eps(eps: float, d: int):
    rng = np.random.default_rng(int(eps * 1e9))
    try:
        for _ in range(100):
            a, b = rng.random(2)
            res = MultiplyTwoReals(bitsize=d).on_classical_vals(**{'a': a, 'b': b})
            assert_allclose(res['result'], a * b, eps)
        print(f'Success! {d=}, {eps=}')
    except AssertionError:
        print(f'Failed! {d=}, {eps=}')

for eps in [1e-3, 1e-4, 1e-5, 1e-6, 1e-7]:
    d = get_bitsize_for_fxp_mul(eps)
    test_multiplication_for_eps(eps, d - 1) # Works for d-1 as well, bounds in the paper are probably loose?

###  Appendix D6: Squaring a real number
Goal: Given a quantum register A with a real number $\kappa$ ($0 \leq \kappa \lt 1$), our goal is to compute an approximation $\widetilde{\gamma}$ of $\gamma = \kappa^2$ s.t. $|\widetilde{\gamma} - \gamma| < \epsilon$

$$
    |\kappa\rangle_{A} |0\rangle_{\text{out}} \rightarrow |\kappa\rangle_{A} |\widetilde{\gamma}\rangle_{\text{out}}
$$


We analyze the error due to fixed point arithmetic for algorithm described in Appendix D6 of https://arxiv.org/abs/2007.07391

In [ ]:
# Squaring a real number
from qualtran.bloqs.arithmetic.multiplication import SquareRealNumber


def get_bitsize_for_fxp_square(eps: float):
    return int(np.ceil(np.log2(1/eps) + np.log2(11/3 + np.log2(1/eps)))) # Equation D36

def test_square_for_eps(eps: float, d: int):
    rng = np.random.default_rng(int(eps * 1e9))
    try:
        for _ in range(100):
            a, = rng.random(1)
            res = SquareRealNumber(bitsize=d).on_classical_vals(a=a)
            assert_allclose(res['result'], a**2, eps)
        print(f'Success! {d=}, {eps=}')
    except AssertionError:
        print(f'Failed! {d=}, {eps=}')


for eps in [1e-3, 1e-4, 1e-5, 1e-6, 1e-7]:
    d = get_bitsize_for_fxp_square(eps) # Need a +1 to make it work for smaller `eps`. Bound in the paper is too tight?
    test_square_for_eps(eps, d)